# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

In [41]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [50]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,median_age_i,median_age_j,population_density_i,population_density_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,38.484,26.235,240.827591,3.950483,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,38.441,18.160,8.881629,84.937007,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,15.214,42.184,25.553880,530.377275,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,30.825,32.014,107.297585,97.915257,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,33.673,43.292,180.584615,65.342441,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,30.699,38.893,121.280071,1575.193750,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [43]:
# Делаю pulled OLS ради интереса
m0 = feols(log(v/v19) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df)
m0 %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.295297   0.023853 12.380068  < 2.2e-16
log(deaths_i)                         0.006162   0.004416  1.395247 1.6472e-01
log(deaths_j)                         0.001219   0.003722  0.327451 7.4372e-01
C8EV_International_travel_controls_i -0.044532   0.011159 -3.990762 9.6932e-05
C8EV_International_travel_controls_j -0.070629   0.010809 -6.534204 6.8081e-10
                                        
(Intercept)                          ***
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i ***
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.999317   Adj. R2: 0.009329

In [44]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
m1 = feols(log(v/v19) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i + j + year,
 data = df)
m1 %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
log(deaths_i)                        -0.015588   0.006061 -2.571959 0.010948
log(deaths_j)                        -0.003195   0.005269 -0.606428 0.545021
C8EV_International_travel_controls_i -0.021174   0.013888 -1.524631 0.129167
C8EV_International_travel_controls_j -0.021157   0.013751 -1.538635 0.125710
                                        
log(deaths_i)                        *  
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.981558     Adj. R2: 0.038294
                 Within R2: 2.777e-4

In [45]:
# Добавляю гетерогенность эффектов во времени
m2 = feols(log(v/v19) ~ year/log(deaths_i) + year/log(deaths_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i + j ,
data = df)
m2 %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                       0.078145   0.077102  1.013525
year2022                                       0.132887   0.088895  1.494875
year2020:log(deaths_i)                        -0.014540   0.006968 -2.086681
year2021:log(deaths_i)                        -0.014148   0.006784 -2.085578
year2022:log(deaths_i)                        -0.021569   0.007546 -2.858254
year2020:log(deaths_j)                        -0.010634   0.006580 -1.616124
year2021:log(deaths_j)                         0.001444   0.006831  0.211366
year2022:log(deaths_j)                         0.003974   0.007462  0.532606
year2020:C8EV_International_travel_controls_i -0.057193   0.025847 -2.212732
year2021:C8EV_International_travel_controls_i -0.026873   0.012924 -2.079314
year2022:C8EV_International

In [46]:
# С кластеризацией ошибок есть значимый эффект и наблюдается гетерогенность во времени (+ направление связи соотносится с описанным в литературе)
etable(m0, m1, m2, vcov = ~i+j, signif.code=signif.code)
# А чем такая кластеризация отличается от i^j?

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2953*** (0.0238),,
4,log(deaths_i),0.0062 (0.0044),-0.0156* (0.0061),
5,log(deaths_j),0.0012 (0.0037),-0.0032 (0.0053),
6,C8EV_International_travel_controls_i,-0.0445*** (0.0112),-0.0212 (0.0139),
7,C8EV_International_travel_controls_j,-0.0706*** (0.0108),-0.0212 (0.0138),
8,year2021,,,0.0781 (0.0771)
9,year2022,,,0.1329 (0.0889)


In [47]:
# При использовании НАС ошибок все эффекты становятся незначимыми
etable(m0, m1, m2, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2953** (0.0131),,
4,log(deaths_i),0.0062 (0.0030),-0.0156 (0.0055),
5,log(deaths_j),0.0012 (0.0028),-0.0032 (0.0057),
6,C8EV_International_travel_controls_i,-0.0445* (0.0070),-0.0212 (0.0122),
7,C8EV_International_travel_controls_j,-0.0706* (0.0072),-0.0212 (0.0120),
8,year2021,,,0.0781 (0.0779)
9,year2022,,,0.1329 (0.0787)


In [48]:
# Всё-таки кластеризовать только по году нельзя, потому что мы в формуле указываем кусочки ошибок которые скоррелированные
    # Пробую кластеризацию ошибок по годам: с теоретической точки зрения это обосновано тем,
    # что все связанные с одной страной наблюдения связаны, значит и во всём графе наблюдения связаны.
    # При этом если нет автокорреляции (учитывая, что у меня 3 периода, это вроде ОК), то кластеризация по годам именно так и делает.
    # Но в статьях пишут, что нельзя использовать кластеризацию ошибок, если меньше чем 50 кластеров...
    # Интересно, что здесь эффект импортёра значимый, а в предыдущей спецификации эффект экспортёра...
etable(m0, m1, m2, vcov = ~i+j+year, signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2953* (0.0334),,
4,log(deaths_i),0.0062 (0.0084),-0.0156 (0.0066),
5,log(deaths_j),0.0012 (0.0036),-0.0032 (0.0036),
6,C8EV_International_travel_controls_i,-0.0445 (0.0179),-0.0212 (0.0117),
7,C8EV_International_travel_controls_j,-0.0706 (0.0288),-0.0212 (0.0139),
8,year2021,,,0.0781 (0.0622)
9,year2022,,,0.1329 (0.0834)


In [49]:
# Не понятно, насколько это осмысленно, но сделал на всякий случай
etable(m0, m1, m2, vcov = ~i^year+j^year, signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2953*** (0.0231),,
4,log(deaths_i),0.0062. (0.0037),-0.0156** (0.0049),
5,log(deaths_j),0.0012 (0.0035),-0.0032 (0.0047),
6,C8EV_International_travel_controls_i,-0.0445*** (0.0115),-0.0212. (0.0109),
7,C8EV_International_travel_controls_j,-0.0706*** (0.0113),-0.0212. (0.0113),
8,year2021,,,0.0781 (0.0624)
9,year2022,,,0.1329. (0.0701)


### Инструментальные переменные на панельных данных

In [21]:
# Pulled TSLS
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
m3 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) 
 ~ 
#  Закомментил инструменты, которые тест Саргана считал эндогенными.
#  median_age_i + median_age_j 
# +
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
#  share_1_members_i + share_1_members_j
#  +
  population_density + population_density_j
  ,
 data = df
 )
m3 %>% summary(vcov = ~i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/9,802).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,739
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.476335   0.065764  7.243076 1.3613e-11
fit_log(deaths_i)                     0.037231   0.071713  0.519160 6.0431e-01
fit_log(deaths_j)                     0.126905   0.053246  2.383395 1.8231e-02
C8EV_International_travel_controls_i -0.070221   0.022920 -3.063776 2.5336e-03
C8EV_International_travel_controls_j -0.064612   0.027831 -2.321550 2.1416e-02
                                        
(Intercept)                          ***
fit_log(deaths_i)                       
fit_log(deaths_j)                    *  
C8EV_International_travel_controls_i ** 
C8EV_Interna

In [19]:
# Только FE года, потому что population_density не меняется во времени (зато экзоенный и релевантный)
m4 = feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year|
 log(deaths_i) + log(deaths_j) ~ 
#  median_age_i + median_age_j 
# +
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
#  share_1_members_i + share_1_members_j
#  +
  population_density + population_density_j
 ,
 data = df
 )
m4 %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/9,802).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                     Estimate Std. Error  t value Pr(>|t|) 
fit_log(deaths_i)                    0.006736   0.047118 0.142953  0.88649 
fit_log(deaths_j)                    0.088905   0.089437 0.994050  0.32158 
C8EV_International_travel_controls_i 0.004323   0.021274 0.203225  0.83920 
C8EV_International_travel_controls_j 0.005673   0.035776 0.158563  0.87420 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.01304     Adj. R2: -0.018103
                Within R2: -0.033871
F-test (1st stage), log(deaths_i): stat = 128.7    , p < 2.2e-16 , on 4 and 56,732 DoF.
F-test (1st stage), log(deaths_j): stat =  91.8    , p < 2.

In [35]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 
# Добавить больше инструментов, которые меняются во времени. Мб плотность населения или медианный возраст брать не за 2019 год а в динамике, 
# чтобы использовать их как инструменты в этой спецификации.
m5 = feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
 ,
 data = df
 )
m5 %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/9,802).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                    -0.019454   0.070691 -0.275205  0.78348 
fit_log(deaths_j)                    -0.109182   0.091360 -1.195076  0.23368 
C8EV_International_travel_controls_i -0.020587   0.029381 -0.700698  0.48443 
C8EV_International_travel_controls_j -0.067796   0.045437 -1.492084  0.13749 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.985502     Adj. R2:  0.030551
                 Within R2: -0.007772
F-test (1st stage), log(deaths_i): stat = 303.4    , p < 2.2e-16 , on 2 and 56,386 DoF.
F-test (1st stage), log(deaths_j): stat = 333.5    , p < 2.2e-16 , on 2

In [36]:
etable(m3, m4, m5, vcov = ~i+j, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.4763*** (0.0658),,
4,log(deaths_i),0.0372 (0.0717),0.0067 (0.0471),-0.0194 (0.0707)
5,log(deaths_j),0.1269* (0.0532),0.0889 (0.0894),-0.1092 (0.0914)
6,C8EV_International_travel_controls_i,-0.0702** (0.0229),0.0043 (0.0213),-0.0206 (0.0294)
7,C8EV_International_travel_controls_j,-0.0646* (0.0278),0.0057 (0.0358),-0.0678 (0.0454)
8,Fixed-Effects:,------------------,---------------,----------------
9,year,No,Yes,Yes


In [37]:
etable(m3, m4, m5, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.4763** (0.0275),,
4,log(deaths_i),0.0372 (0.0241),0.0067 (0.0298),-0.0194 (0.0604)
5,log(deaths_j),0.1269* (0.0245),0.0889 (0.0354),-0.1092 (0.0549)
6,C8EV_International_travel_controls_i,-0.0702* (0.0102),0.0043 (0.0138),-0.0206 (0.0276)
7,C8EV_International_travel_controls_j,-0.0646* (0.0102),0.0057 (0.0150),-0.0678 (0.0272)
8,Fixed-Effects:,-----------------,---------------,----------------
9,year,No,Yes,Yes


In [51]:
    # Всё ещё странно использовать кластеризацию по году... 
    # etable(m3, m4, vcov = function(x) vcovHC(x, method="arellano", type="HC1"), signif.code = signif.code) Не сработало
etable(m3, m4, m5, vcov = ~i+j+year, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.4763 (0.1869),,
4,log(deaths_i),0.0372 (0.0943),0.0067 (0.0420),-0.0194 (0.0600)
5,log(deaths_j),0.1269 (0.0504),0.0889 (0.1137),-0.1092 (0.0722)
6,C8EV_International_travel_controls_i,-0.0702. (0.0219),0.0043 (0.0136),-0.0206 (0.0221)
7,C8EV_International_travel_controls_j,-0.0646 (0.0243),0.0057 (0.0283),-0.0678 (0.0430)
8,Fixed-Effects:,-----------------,---------------,----------------
9,year,No,Yes,Yes


In [ ]:
# Так вроде не очень корректно делать, если мы верим в возможную автокорреляцию
etable(m3, m4, m5, vcov = ~i^year+j^year, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.4763*** (0.0647),,
4,log(deaths_i),0.0372 (0.0563),0.0067 (0.0405),-0.0194 (0.0555)
5,log(deaths_j),0.1269** (0.0477),0.0889 (0.0687),-0.1092 (0.0724)
6,C8EV_International_travel_controls_i,-0.0702** (0.0215),0.0043 (0.0187),-0.0206 (0.0243)
7,C8EV_International_travel_controls_j,-0.0646** (0.0237),0.0057 (0.0297),-0.0678. (0.0363)
8,Fixed-Effects:,------------------,---------------,-----------------
9,year,No,Yes,Yes


In [ ]:
# С помощью fixest::demean можно отрисовать scatterplot, который обосновывает необходимость использовать фиксированные эффекты

## Три пространственные регрессии (как в курсовой)

Но в такую спецификацию нельзя добавить фиксированные эффекты, поэтому почти наверное без инструментов будет смещение из-за пропущенных переменных

In [40]:
# Без инструментов.
m_2020 = feols(log(v/v19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v/v19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v/v19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 5,531).

NOTE: 4,126 observations removed because of NA and infinite values (RHS: 4,126).

NOTE: 3,911 observations removed because of NA and infinite values (RHS: 3,911).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 18,391
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           0.025597   0.046849  0.546368 0.5848197
log(deaths_i)                         0.004091   0.003260  1.254760 0.2095820
log(deaths_j)                        -0.009842   0.003226 -3.051141 0.0022830
C8EV_International_travel_controls_i -0.016298   0.013407 -1.215616 0.2241469
C8EV_International_travel_controls_j -0.034239   0.012831 -2.668548 0.0076247
                                        
(Intercept)                             
log(deaths_i)                           
log(deaths_j)                        ** 
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.86853   Adj. R2: 8.875e-4

OLS estimation, Dep. Var.: log(v/v19)
Observations: 19,432
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.121186   0.035212  3.441616 0.00057947
log(deaths_i)                         0.003676   0.004665  0.787934 0.43074466
log(deaths_j)                        -0.004846   0.004284 -1.131245 0.25796589
C8EV_International_travel_controls_i -0.002579   0.009918 -0.260020 0.79485144
C8EV_International_travel_controls_j -0.004681   0.010183 -0.459701 0.64573592
                                        
(Intercept)                          ***
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.0266   Adj. R2: -8.166e-5

OLS estimation, Dep. Var.: log(v/v19)
Observations: 18,916
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.239995   0.019842 12.095399  < 2.2e-16
log(deaths_i)                        -0.003994   0.004750 -0.840924 0.40040089
log(deaths_j)                        -0.001500   0.004216 -0.355718 0.72205574
C8EV_International_travel_controls_i  0.021464   0.019187  1.118691 0.26328611
C8EV_International_travel_controls_j -0.063888   0.017143 -3.726715 0.00019456
                                        
(Intercept)                          ***
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.07671   Adj. R2: 7.389e-4

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0256 (0.0469),0.1212*** (0.0352),0.2400*** (0.0198)
4,log(deaths_i),0.0041 (0.0033),0.0037 (0.0047),-0.0040 (0.0047)
5,log(deaths_j),-0.0098** (0.0032),-0.0048 (0.0043),-0.0015 (0.0042)
6,C8EV_International_travel_controls_i,-0.0163 (0.0134),-0.0026 (0.0099),0.0215 (0.0192)
7,C8EV_International_travel_controls_j,-0.0342** (0.0128),-0.0047 (0.0102),-0.0639*** (0.0171)
8,____________________________________,__________________,__________________,___________________
9,S.E. type,Heteroskedas.-rob.,Heteroskedas.-rob.,Heteroskedast.-rob.


In [17]:
# С инструментами оценки коэффициентов похожи на оценки на панельных данных (очень приблизительно).
# Использую ошибки hc1
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
# Почему экзогенные в панельной спецификации инструменты эндогенны в пространственной?
m_2020 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 median_age_i + median_age_j + population_density + population_density_j,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 median_age_i + median_age_j + population_density + population_density_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 median_age_i + median_age_j + population_density + population_density_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)
etable(m_2020, m_2021, m_2022, vcov = ~i+j, signif.code = signif.code)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 3,419, IV: 3,381/450).

NOTE: 4,126 observations removed because of NA and infinite values (RHS: 3,290, IV: 1,557/425).

NOTE: 3,911 observations removed because of NA and infinite values (RHS: 3,093, IV: 1,129/387).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 18,391
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           0.026734   0.046711  0.572332 0.5671042
fit_log(deaths_i)                     0.009457   0.005617  1.683633 0.0922696
fit_log(deaths_j)                    -0.012174   0.005339 -2.280292 0.0226018
C8EV_International_travel_controls_i -0.014747   0.013422 -1.098734 0.2718984
C8EV_International_travel_controls_j -0.034874   0.012910 -2.701355 0.0069121
                                        
(Intercept)                             
fit_log(deaths_i)                    .  
fit_log(deaths_j)                    *  
C8EV_International_travel_controls_i    
C8EV_International_travel_cont

TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 19,432
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.121777   0.035106  3.468841 0.00052384
fit_log(deaths_i)                     0.000992   0.007856  0.126251 0.89953424
fit_log(deaths_j)                    -0.000903   0.007144 -0.126359 0.89944937
C8EV_International_travel_controls_i -0.002279   0.009937 -0.229317 0.81862473
C8EV_International_travel_controls_j -0.005017   0.010201 -0.491836 0.62284093
                                        
(Intercept)                          ***
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i    
C8EV_International_trave

TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 18,916
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.236611   0.019822 11.936839  < 2.2e-16
fit_log(deaths_i)                    -0.010381   0.005554 -1.868906 0.06165139
fit_log(deaths_j)                    -0.009636   0.005040 -1.911843 0.05591137
C8EV_International_travel_controls_i  0.013399   0.019168  0.699029 0.48454223
C8EV_International_travel_controls_j -0.074282   0.017385 -4.272661 0.00001941
                                        
(Intercept)                          ***
fit_log(deaths_i)                    .  
fit_log(deaths_j)                    .  
C8EV_International_travel_controls_i    
C8EV_International_trave

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0267 (0.0467),0.1218*** (0.0351),0.2366*** (0.0198)
4,log(deaths_i),0.0095. (0.0056),0.0010 (0.0079),-0.0104. (0.0056)
5,log(deaths_j),-0.0122* (0.0053),-0.0009 (0.0071),-0.0096. (0.0050)
6,C8EV_International_travel_controls_i,-0.0147 (0.0134),-0.0023 (0.0099),0.0134 (0.0192)
7,C8EV_International_travel_controls_j,-0.0349** (0.0129),-0.0050 (0.0102),-0.0743*** (0.0174)
8,____________________________________,__________________,__________________,___________________
9,S.E. type,Heteroskedas.-rob.,Heteroskedas.-rob.,Heteroskedast.-rob.


,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0267 (0.0658),0.1218* (0.0492),0.2366*** (0.0428)
4,log(deaths_i),0.0095 (0.0063),0.0010 (0.0109),-0.0104 (0.0100)
5,log(deaths_j),-0.0122. (0.0073),-0.0009 (0.0102),-0.0096 (0.0089)
6,C8EV_International_travel_controls_i,-0.0147 (0.0180),-0.0023 (0.0139),0.0134 (0.0447)
7,C8EV_International_travel_controls_j,-0.0349* (0.0176),-0.0050 (0.0135),-0.0743. (0.0420)
8,____________________________________,_________________,________________,__________________
9,S.E.: Clustered,by: i & j,by: i & j,by: i & j
